转载：[莫烦Python之保存提取](https://morvanzhou.github.io/tutorials/machine-learning/torch/3-04-save-reload/)和[PyTorch预训练](https://zhuanlan.zhihu.com/p/25980324)和[pytorch finetune模型](https://www.jianshu.com/p/19957bd2bcb7)

# 1 要点
训练好了一个模型, 我们当然想要保存它, 留到下次要用的时候直接提取直接用, 这就是这节的内容啦. 我们用回归的神经网络举例实现保存提取.

# 2 保存
我们快速地建造数据, 搭建网络:

In [7]:
import torch
from torch.autograd import Variable

torch.manual_seed(1)    # reproducible

# 假数据
x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
y = x.pow(2) + 0.2*torch.rand(x.size())  # noisy y data (tensor), shape=(100, 1)

x = Variable(x)
y = Variable(y)

def save():
    # 建网络
    net1 = torch.nn.Sequential(
        torch.nn.Linear(1, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1)
    )
    optimizer = torch.optim.SGD(net1.parameters(), lr=0.5)
    loss_func = torch.nn.MSELoss()

    # 训练
    for t in range(100):
        prediction = net1(x)
        loss = loss_func(prediction, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    torch.save(net1, 'net.pkl')  # 保存整个网络
    torch.save(net1.state_dict(), 'net_params.pkl')   # 只保存网络中的参数 (速度快, 占内存少)

In [8]:
save()

In [9]:
ls

01_tensor_tutorial.ipynb           05_data_parallel_tutorial.ipynb  net.pkl
02_autograd_tutorial.ipynb         06_save_load_model.ipynb         nohup.out
03_neural_networks_tutorial.ipynb  data/
04_cifar10_tutorial.ipynb          net_params.pkl


In [10]:
ls -la

total 1452
drwxr-xr-x 10 rqchen rqchen    4096 Aug 13 13:05 ./
drwxr-xr-x 29 root   root      4096 Aug  5 11:34 ../
-rw-rw-r--  1 rqchen rqchen   11758 Aug  9 11:36 01_tensor_tutorial.ipynb
-rw-rw-r--  1 rqchen rqchen   17377 Aug  9 11:01 02_autograd_tutorial.ipynb
-rw-rw-r--  1 rqchen rqchen   42977 Aug  9 12:42 03_neural_networks_tutorial.ipynb
-rw-rw-r--  1 rqchen rqchen 1274263 Aug  9 23:24 04_cifar10_tutorial.ipynb
-rw-rw-r--  1 rqchen rqchen   16039 Aug 10 20:18 05_data_parallel_tutorial.ipynb
-rw-rw-r--  1 rqchen rqchen   13888 Aug 13 13:04 06_save_load_model.ipynb
-rw-------  1 rqchen rqchen     828 Aug  9 16:57 .bash_history
-rw-r--r--  1 rqchen rqchen     220 Aug  5 11:34 .bash_logout
-rw-r--r--  1 rqchen rqchen    3771 Aug  5 11:34 .bashrc
drwx------  5 rqchen rqchen    4096 Aug  9 10:20 .cache/
drwxrwxr-x  3 rqchen rqchen    4096 Aug  9 10:20 .config/
drwxrwxr-x  3 rqchen rqchen    4096 Aug  9 12:55 data/
drwxrwxr-x  2 rqchen rqchen    4096 Aug 13 12:56 .ipynb_checkpoints/


# 3 提取网络
这种方式将会提取整个神经网络, 网络大的时候可能会比较慢.

In [16]:
def restore_net():
    # restore entire net1 to net2
    net2 = torch.load('net.pkl')
    prediction = net2(x)

# 4 只提取网络参数
这种方式将会提取所有的参数, 然后再放到你的新建网络中.

In [11]:
def restore_params():
    # 新建 net3
    net3 = torch.nn.Sequential(
        torch.nn.Linear(1, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1)
    )

    # 将保存的参数复制到 net3
    net3.load_state_dict(torch.load('net_params.pkl'))
    prediction = net3(x)

# 5 显示结果
调用上面建立的几个功能, 然后出图.

In [18]:
# 保存 net1 (1. 整个网络, 2. 只有参数)
save()

# 提取整个网络
restore_net()

# 提取网络参数, 复制到新网络
restore_params()

# 6 加载部分预训练模型
其实大多数时候我们需要根据我们的任务调节我们的模型，所以很难保证模型和公开的模型完全一样，但是预训练模型的参数确实有助于提高训练的准确率，为了结合二者的优点，就需要我们加载部分预训练模型。

In [5]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo

In [6]:
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [7]:
pretrained_dict = model_zoo.load_url(model_urls['resnet152'])

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/rqchen/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:07<00:00, 32497544.08it/s]


In [10]:
pretrained_dict.viewkeys

<bound method OrderedDict.viewkeys of OrderedDict([(u'conv1.weight', 
(0 ,0 ,.,.) = 
  4.7132e-07  6.3123e-07  6.1915e-07  ...   2.9313e-07  2.1123e-07  1.3036e-07
  4.8263e-07  7.1548e-07  7.1251e-07  ...   3.0581e-07  2.6611e-07  2.3413e-07
  4.9888e-07  6.3326e-07  6.1920e-07  ...   1.2629e-07  1.8429e-07  2.0732e-07
                 ...                   ⋱                   ...                
  5.5013e-07  3.1735e-07  4.1098e-07  ...   3.1079e-07  3.4928e-07  3.4718e-07
  6.2982e-07  4.0325e-07  3.4432e-07  ...   4.8297e-07  6.4529e-07  5.4214e-07
  7.1402e-07  5.0883e-07  4.4785e-07  ...   6.2946e-07  6.5617e-07  5.0979e-07

(0 ,1 ,.,.) = 
  5.0878e-07  6.8802e-07  6.1782e-07  ...   2.2142e-07  2.1541e-07  1.8464e-07
  4.2393e-07  6.5220e-07  6.2894e-07  ...   2.8318e-07  2.5690e-07  2.3177e-07
  4.6649e-07  6.4230e-07  6.2854e-07  ...   1.3226e-07  2.2451e-07  2.1060e-07
                 ...                   ⋱                   ...                
  4.9365e-07  2.8871e-07  3.92

In [ ]:
model_dict = model.state_dict()
# 将pretrained_dict里不属于model_dict的键剔除掉
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 更新现有的model_dict
model_dict.update(pretrained_dict)
# 加载我们真正需要的state_dict
model.load_state_dict(model_dict)

因为需要剔除原模型中不匹配的键，也就是层的名字，所以我们的新模型改变了的层需要和原模型对应层的名字不一样，比如：resnet最后一层的名字是fc(PyTorch中)，那么我们修改过的resnet的最后一层就不能取这个名字，可以叫fc_

## 6.1 如果模型的key值和在大数据集上训练时的key值是一样的
我们可以通过下列算法进行读取模型

In [ ]:
model_dict = model.state_dict()

pretrained_dict = torch.load(model_path)
 # 1. filter out unnecessary keys
diff = {k: v for k, v in model_dict.items() if \
            k in pretrained_dict and pretrained_dict[k].size() == v.size()}
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and model_dict[k].size() == v.size()}
pretrained_dict.update(diff)
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
# 3. load the new state dict
model.load_state_dict(model_dict)

## 6.2 如果模型的key值和在大数据集上训练时的key值是不一样的，但是顺序是一样的

In [ ]:
model_dict = model.state_dict()

pretrained_dict = torch.load(model_path)
keys = []
for k,v in pretrained_dict.items():
    keys.append(k)
i = 0
for k,v in model_dict.items():
    if v.size() == pretrained_dict[keys[i]].size():
        print(k, ',', keys[i])
         model_dict[k]=pretrained_dict[keys[i]]
    i = i + 1
model.load_state_dict(model_dict)

## 6.3 如果模型的key值和在大数据集上训练时的key值是不一样的，但是顺序是也不一样的
自己找对应关系，一个key对应一个key的赋值


# 7 微改基础模型预训练
对于改动比较大的模型，我们可能需要自己实现一下再加载别人的预训练参数。但是，对于一些基本模型PyTorch中已经有了，而且我只想进行一些小的改动那么怎么办呢？难道我又去实现一遍吗？当然不是。

我们首先看看怎么进行微改模型。

## 7.1 微改基础模型
PyTorch中的torchvision里已经有很多常用的模型了，可以直接调用：

* AlexNet
* VGG
* ResNet
* SqueezeNet
* DenseNet

In [2]:
import torchvision.models as models
resnet18 = models.resnet18()
alexnet = models.alexnet()
squeezenet = models.squeezenet1_0()
densenet = models.densenet161()

但是对于我们的任务而言有些层并不是直接能用，需要我们微微改一下，比如，resnet最后的全连接层是分1000类，而我们只有21类；又比如，resnet第一层卷积接收的通道是3， 我们可能输入图片的通道是4，那么可以通过以下方法修改：

In [ ]:
resnet.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet.fc = nn.Linear(2048, 21)

## 7.2 简单预训练
模型已经改完了，接下来我们就进行简单预训练吧。 
我们先从torchvision中调用基本模型，加载预训练模型，然后，重点来了，**将其中的层直接替换为我们需要的层即可：**

In [ ]:
resnet = torchvision.models.resnet152(pretrained=True)
# 原本为1000类，改为10类
resnet.fc = torch.nn.Linear(2048, 10)

其中使用了pretrained参数，会直接加载预训练模型，内部实现和前文提到的加载预训练的方法一样。因为是先加载的预训练参数，相当于模型中已经有参数了，所以替换掉最后一层即可。